# Gaussian Mixture Models (Clustering)

**Context:** Sometimes our data contains hidden structure---structure that we'd like to uncover in order to answer some scientific question. 

**Challenge:** TODO

**Outline:** TODO

**Data:** TODO

## Latent Variable Models

TODO:
* Introduce concept of latent variables
* Introduce GMMs as a specific model and explain its usefulness (also explain its relation to "clustering") <-- introduce its generative process and its DGM
* Exercise: (a) describe scenarios and as students to say which variable is latent, (b) ask them to come up with scenarios with latent variables
* Highlight the problem: what's the MLE for this model?

## The Law of Total Probability (Discrete)

**Definition.** Suppose you have two random variables, $A$ and $B$ with joint distribution, and suppose that $A$ is discrete. Then the law of total probability says we can compute the marginal $p_B(b)$ from the joint $p_{A, B}(a, b)$ as follows:
\begin{align}
p_B(b) &= \sum\limits_{a \in S_A} p_{A, B}(a, b)
\end{align}

**Intuition.**

## Maximum Likelihood for GMMs

## GMMs in `NumPyro`

* Ask to generate samples
* Ask to infer clusters